# Building Fast Queries On a CSV

In this project, we are given a csv containing laptop data:

**ID:** A unique identifier for the laptop.  
**Company:** The name of the company that produces the laptop.  
**Product:** The name of the laptop.  
**TypeName:** The type of laptop.  
**Inches:** The size of the screen in inches.  
**ScreenResolution:** The resolution of the screen.  
**CPU**: The laptop CPU.  
**RAM**: The amount of RAM in the laptop.  
**Memory**: The size of the hard drive.  
**GPU**: The graphics card name.  
**OpSys**: The name of the operating system.  
**Weight**: The laptop weight.  
**Price**: The price of the laptop.  

We want to build a way to answer certain questions about our laptop inventory in an efficient manner.

## Objectives
1. Analyze the time and space complexity of an algorithm.
2. Preprocesse data to significantly speed-up an algorithm.
3. Sort data and efficiently search sorted data.

## The Dataset

In [1]:
import csv

with open("laptops.csv") as file:
    file_contents = list(csv.reader(file))
    header = file_contents[0]
    rows = file_contents[1:]

print("Header is:", header)
print("\n")
print("First five rows:", rows[:5])

## Inventory Class

In [2]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            file_contents = list(csv.reader(file))
            
        self.header = file_contents[0]
        self.rows = file_contents[1:]
        
        for row in self.rows:
            row[-1] = int(row[-1])
            
# Testing Inventory Class
laptop_inventory = Inventory('laptops.csv')
print("Laptop Inventory Header:", laptop_inventory.header)
print("\n")
print("Number of rows in Inventory:", len(laptop_inventory.rows))